# Navodila
Poženi naslednji ukaz za generacijo datoteke `messages2.py`

```sh
python /usr/local/lib/python2.7/dist-packages/asn1ate/pyasn1gen.py messages.asn > messages.py
```

In [1]:
from pyasn1.type import univ
from pyasn1.type import char
from pyasn1.type import tag
from pyasn1.type import namedtype
from pyasn1.type import constraint
from pyasn1.type import namedval
from pyasn1.codec.ber import encoder
from pyasn1.codec.ber import decoder
from trustmessages import *
import base64
import itertools

In [2]:
encode = lambda m: base64.b64encode(encoder.encode(m))
users = itertools.cycle(["a@x.com", "b@x.com", "c@x.com"])
services = itertools.cycle(["buyer", "seller", "letter", "renter"])
quantitative = itertools.cycle(xrange(1, 6))
qualitative = itertools.cycle(["distrust", "neutral", "trust"])

In [3]:
r = Assessment()
r["tms"] = univ.ObjectIdentifier((1, 1, 1))
r["source"] = next(users)
r["target"] = next(users)
r["service"] = next(services)
r["date"] = 100
val = next(quantitative)
r["value"] = encoder.encode(univ.Integer(val))

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r.prettyPrint() == decoded.prettyPrint())
assert(decoder.decode(decoded["value"], asn1Spec=univ.Integer())[0] == val)
print encode(r)

aSMGAikBEwdhQHguY29tEwdiQHguY29tEwVidXllcgIBZAIBAQ==


In [4]:
r = Assessment()
r["tms"] = univ.ObjectIdentifier((2, 2, 2))
r["source"] = next(users)
r["target"] = next(users)
r["service"] = next(services)
r["date"] = 100
val = next(qualitative)
r["value"] = encoder.encode(char.PrintableString(val))

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r.prettyPrint() == decoded.prettyPrint())
assert(decoder.decode(decoded["value"], asn1Spec=char.PrintableString())[0] == val)
print encode(r)

aSsGAlICEwdjQHguY29tEwdhQHguY29tEwZzZWxsZXICAWQTCGRpc3RydXN0


In [5]:
a_req = AssessmentRequest()
a_req["rid"] = 1
sq = Query()
sq["cmp"] = Comparison()
sq["cmp"]["op"] = "ge"
sq["cmp"]["value"] = Value()
sq["cmp"]["value"]["date"] = 80
a_req["query"] = sq

decoded, _ = decoder.decode(encoder.encode(a_req), asn1Spec=Message())
assert(a_req.prettyPrint() == decoded.getComponent().prettyPrint())
print encode(a_req)

ZgsCAQFkBgoBBUIBUA==


In [6]:
a_res = AssessmentResponse()
a_res["rid"] = 1
a_res["response"] = univ.SequenceOf(componentType=Assessment())

for i in xrange(2):
    a = Assessment()
    a["tms"] = SystemIdentity((1, 1, 1))
    a["source"] = next(users)
    a["target"] = next(users)
    a["service"] = next(services)
    a["date"] = 1000
    a["value"] = encoder.encode(univ.Integer(5))
    a_res["response"].setComponentByPosition(i, a)

data, _ = decoder.decode(encoder.encode(a_res), asn1Spec=Message())
assert(data.getComponent() == a_res)
assert(data.getComponent().prettyPrint() == a_res.prettyPrint())
print encode(a_res)

Z1MCAQEwTmklBgIpARMHYkB4LmNvbRMHY0B4LmNvbRMGbGV0dGVyAgID6AIBBWklBgIpARMHYUB4LmNvbRMHYkB4LmNvbRMGcmVudGVyAgID6AIBBQ==


In [7]:
t_req = TrustRequest()

t_req["rid"] = 5000

sq = Query()
sq["cmp"] = Comparison()
sq["cmp"]["op"] = "ge"
sq["cmp"]["value"] = Value()
sq["cmp"]["value"]["date"] = 80
t_req["query"] = sq

data, _ = decoder.decode(encoder.encode(t_req), asn1Spec=Message())
assert(data.getComponent() == t_req)
assert(data.getComponent().prettyPrint() == t_req.prettyPrint())
print encode(t_req)

YgwCAhOIZAYKAQVCAVA=


In [8]:
t_res = TrustResponse()
t_res["rid"] = 70000
t_res["response"] = univ.SequenceOf(componentType=Trust())

for i in xrange(2):
    t = Trust()
    t["tms"] = univ.ObjectIdentifier((1, 1, 1))
    t["target"] = next(users)
    t["service"] = next(services)
    t["date"] = 2000
    t["value"] = encoder.encode(char.PrintableString(next(qualitative)))
    t_res["response"].setComponentByPosition(i, t)

data, _ = decoder.decode(encoder.encode(t_res), asn1Spec=Message())

assert(data.getComponent() == t_res)
assert(data.getComponent().prettyPrint() == t_res.prettyPrint())
print encode(t_res)

Y0wCAwERcDBFaCEGAikBEwdjQHguY29tEwVidXllcgICB9ATB25ldXRyYWxoIAYCKQETB2FAeC5jb20TBnNlbGxlcgICB9ATBXRydXN0


In [9]:
f_req = FormatRequest()
data, _ = decoder.decode(encoder.encode(f_req), asn1Spec=Message())
assert(data.getComponent() == f_req)
assert(data.getComponent().prettyPrint() == f_req.prettyPrint())
print encode(f_req)

QAA=


In [10]:
f_res = FormatResponse()
f_res["tms"] = SystemIdentity((1, 2, 3))
f_res["assessment"] = char.PrintableString("Here be an ASN.1 spec for assessment values")
f_res["trust"] = char.PrintableString("Here be an ASN.1 spec for trust values")

data, _ = decoder.decode(encoder.encode(f_res), asn1Spec=Message())
assert(data.getComponent() == f_res)
assert(data.getComponent().prettyPrint() == f_res.prettyPrint())
print encode(f_res)

YVkGAioDEytIZXJlIGJlIGFuIEFTTi4xIHNwZWMgZm9yIGFzc2Vzc21lbnQgdmFsdWVzEyZIZXJlIGJlIGFuIEFTTi4xIHNwZWMgZm9yIHRydXN0IHZhbHVlcw==


In [11]:
e = Fault()
e["value"] = "invalid-parameters"
e["message"] = "something went wrong!"

data, _ = decoder.decode(encoder.encode(e), asn1Spec=Message())

assert(data.getComponent() == e)
assert(data.getComponent().prettyPrint() == e.prettyPrint())
print encode(e)

ahoKAQATFXNvbWV0aGluZyB3ZW50IHdyb25nIQ==


In [12]:
def p(e):
    print [ord(i) for i in encoder.encode(e)]

In [13]:
e1 = Fault()
e1["value"] = "invalid-parameters"
e1["message"] = "a"

e2 = Fault()
e2["value"] = "internal-error"
e2["message"] = "b"

p(e1)
p(e2)


[106, 6, 10, 1, 0, 19, 1, 97]
[106, 6, 10, 1, 1, 19, 1, 98]
